In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [2]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(dev_users_list[:5])
print(len(test_users_list))
print(test_users_list[:5])    

3000
['#d6866a498157771069fdf15361cb012b', '#f963fb8c5d9d14d503fc4e80bd8617b4', '#87a6479c91e4276374378f1d28eb307c', '#677e984e245b344f61dc5d3cc1f352c8', '#519f45eb14e4807e8714fb7e835463eb']
5000
['#7ee14df8642a7925b1465ff5c89efe5b', '#8420b9385b282028eebf1ad6b4a221c0', '#c9b31d8b64357f5854b1ba55b32eb6d3', '#9bb1e13b5481fa3737af20870b25c723', '#37d5f99a7f12c9ba90c4e2ac92e54ab6']


# Load Article list

In [3]:
with open(directory+'article_by_user.json', 'rb') as f:
    article_by_nontestuser = json.load(f)
with open(directory+'article_by_testuser.json', 'rb') as f:
    article_by_testuser = json.load(f)

In [4]:
print(len(article_by_nontestuser))
print(len(article_by_testuser))

301222
5000


In [5]:
# Load Contents_textrank 
with open(directory+'contents_textrank.json', 'rb') as f:
    contents_textrank = json.load(f)

# Article id 로 Word2Vec


### [1] make values & corpus

In [6]:
import time
start_time = time.time()
articles = []
corpus = []

for values in article_by_testuser.values():
    for x in values:
        articles.append(x)
        if(len(values) > 2): # 개수는 더 늘려도 될듯
            corpus.append(x)
        
    if(len(values) != 0):
        for i in range(8):
            corpus.append('eos')
            
for values in article_by_nontestuser.values():
    for x in values:
        articles.append(x)
        if(len(values) > 2):
            corpus.append(x)
    
    if(len(values) != 0):
        for i in range(8):
            corpus.append('eos')
        
print ("%s sec"%(time.time() - start_time))
print('article_ : ',len(articles))

9.391512632369995 sec
article_ :  22105708


### [2] Make dictionary

In [7]:
# make dictionary
# generate dataset, dictionary for word
start_time = time.time()
word_to_id = {}
id_to_word = {}

for word in articles:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

# 공백 표시할 어휘 : eos
word_to_id['eos'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'eos'
# vocab에 없는 단어는 UNK로 표시
word_to_id['UNK'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'UNK'

print ("%s sec"%(time.time() - start_time))

5.828672885894775 sec


In [8]:
print(len(word_to_id))

505842


### [3] corpus real

- recommend 시 필요없음

In [14]:
corpus = [word_to_id[w] for w in corpus]
print(len(corpus))

24461361


### [4] Model 

- recommend 시 필요없음

In [19]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import collections
def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    for i in range(batch_size // num_skips):

        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index

In [17]:
# Step 4: skip-gram 모델 구축
import math
np.random.seed(1)
tf.set_random_seed(1)
vocabulary_size =(len(word_to_id))
batch_size = 128        # 일반적으로 16 <= batch_size <= 512
embedding_size = 128    # embedding vector 크기
skip_window = 4         # target 양쪽의 단어 갯수
num_skips = 8          # 컨텍스트로부터 생성할 레이블 갯수

valid_size = 5     # 유사성을 평가할 단어 집합 크기
valid_window = 100  # 앞쪽에 있는 분포들만 뽑기 위한 샘플
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # negative 샘플링 갯수

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

truncated = tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size))
nce_weights = tf.Variable(truncated)
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# embeddings 벡터.
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# 배치 데이터에 대해 NCE loss 평균 계산
nce_loss = tf.nn.nce_loss(weights=nce_weights,
                          biases=nce_biases,
                          labels=train_labels,
                          inputs=embed,
                          num_sampled=num_sampled,
                          num_classes=vocabulary_size)
loss = tf.reduce_mean(nce_loss)

# SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

# 유사도를 계산하기 위한 모델. 학습 모델은 optimizer까지 구축한 걸로 종료.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [20]:
# Step 5: skip-gram 모델 학습
# 4시 4분 시작
num_steps = 1000001
data = corpus
ordered_words = list(word_to_id.keys())
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    average_loss, data_index = 0, 0
    for step in range(num_steps):
        batch_inputs, batch_labels, data_index = generate_batch(data, batch_size, num_skips, skip_window, data_index)

        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        # 마지막 10000번에 대한 평균 loss 표시
        if step % 10000 == 0:
            if step > 0:
                average_loss /= 10000
            print('Average loss at step {} : {}'.format(step, average_loss))
            average_loss = 0
        
        # 10만번째마다 valid size만큼 sim 계산
        if step % 100000 == 0:
            sim = similarity.eval()         # (16, vocab_size)

            for i in range(valid_size):
                valid_word = ordered_words[valid_examples[i]]

                top_k = 8
                nearest = sim[i].argsort()[-top_k - 1:-1][::-1]
                log_str = ', '.join([ordered_words[k] for k in nearest])
                print('Nearest to {}: {}'.format(valid_word, log_str))

    final_embeddings = normalized_embeddings.eval()
    saver = tf.train.Saver()
    saver.save(session,'keyword_embedding_model')

Average loss at step 0 : 394.24151611328125
Nearest to @yuhaaustin_45: @vitmania86_232, @sarkyuday_19, @rhee1957_85, @jonaspark_48, @cornerstool_354, @prestigegorilla_282, @bongsub-kim_19, @cym8930_77
Nearest to @yuhaaustin_11: @myfriendjesus_31, @sunsutu_582, @jeda502_43, @popin-boy_4, @litteratus_27, @4cheol_39, @browne_3, @psychiatricnews_39
Nearest to @prestigegorilla_214: @myungsunkim_260, @dreamisme80_330, @quidlesf_48, @doyeonsunim_307, @parksun74_690, @iammystic_29, @hj1003_244, @gangbaram_67
Nearest to @yuhaaustin_52: @meether_15, @onestepculture_215, @dbr_58, @hj5098_20, @sweetmeen_7, @oetc21_47, @liha_98, @transpeace_4
Nearest to @ladybob_30: @simplestories_379, @a-platform_13, @lovelyming_44, @duk-hyun_37, @kellyluvmore_3, @junatul_96, @topofk_28, @aa3062545_113
Average loss at step 10000 : 197.64329766349792
Average loss at step 20000 : 124.82415936956406
Average loss at step 30000 : 94.77116870567798
Average loss at step 40000 : 76.29626282596588
Average loss at step 5000

In [42]:
np.save(directory+'article_embedding_matrix',final_embeddings)

# Recommend

In [ ]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [9]:
article_embeddings_matrix =  np.load(directory+'article_embedding_matrix.npy')

In [60]:
import pickle

with open(directory+'article_embedding_matrix_1', 'wb') as f:
    pickle.dump(article_embeddings_matrix, f)

In [10]:
print(article_embeddings_matrix.shape)
print(type(article_embeddings_matrix))
print(article_embeddings_matrix.dtype)

(505842, 128)
<class 'numpy.ndarray'>
float32


In [11]:
article_data = list(word_to_id.keys())
len(article_data)

505842

In [49]:
tf.reset_default_graph() 
embeddings_article = tf.get_variable("artcle_matrix",initializer = article_embeddings_matrix)
norm_article = tf.sqrt(tf.reduce_sum(tf.square(embeddings_article),1,keep_dims =True))
normalized_embeddings_article = embeddings_article / norm_article

embeddings_article_vector = tf.get_variable("article_vector",initializer = article_embeddings_matrix[0]) ## will be transposed
embeddings_article_vector = tf.reshape(embeddings_article_vector,(128,1),)
norm_article_t = tf.sqrt(tf.reduce_sum(tf.square(embeddings_article_vector),1,keep_dims =True))
normalized_embeddings_article_t = embeddings_article_vector / norm_article_t


similarity = tf.matmul(normalized_embeddings_article, normalized_embeddings_article_t)

In [50]:
top_k = 10
sim_article = tf.placeholder(tf.int32, shape=[len(article_data),top_k])

In [54]:
def cos_similarity(x,y,eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2))+eps)
    ny = y / (np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx,ny)

In [55]:
start_time = time.time()
cos_sim = []
for i in range(len(article_data)):
    cos_sim.append(cos_similarity(article_embeddings_matrix[i],article_embeddings_matrix[0]))
    cos_s = np.asarray(cos_sim)
    cos_sim = cos_s.argsort()[-top_k-1:-1][::-1].tolist()
print ("%s sec"%(time.time() - start_time))

14.87090516090393 sec


In [57]:
start_time = time.time()
cos_sim = []
for i in range(50000):
    cos_sim.append(cos_similarity(article_embeddings_matrix[i],article_embeddings_matrix[0]))
    cos_s = np.asarray(cos_sim)
    cos_sim = cos_s.argsort()[-top_k-1:-1][::-1].tolist()
print ("%s sec"%(time.time() - start_time))

1.4791829586029053 sec


In [53]:
start_time = time.time()
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    sim = similarity.eval()
    sim_top_10 = sim.argsort()[-top_k-1:-1][::-1].tolist()
print ("%s sec"%(time.time() - start_time))

8.33065676689148 sec


In [17]:
tf.reset_default_graph() 

In [24]:
article_embeddings_matrix[0].reshape(128,1).shape

(128, 1)

In [29]:
def cos_loop_spatial(matrix, vector):
    """
    Calculating pairwise cosine distance using a common for loop with the numpy cosine function.
    """
    neighbors = []
    for row in range(matrix.shape[0]):
        neighbors.append(scipy.spatial.distance.cosine(vector, matrix[row,:]))
    return neighbors
def cos_loop(matrix, vector):
    """
    Calculating pairwise cosine distance using a common for loop with manually calculated cosine value.
    """
    neighbors = []
    for row in range(matrix.shape[0]):
        vector_norm = np.linalg.norm(vector)
        row_norm = np.linalg.norm(matrix[row,:])
        cos_val = vector.dot(matrix[row,:]) / (vector_norm * row_norm)
        neighbors.append(cos_val)
    return neighbors

In [58]:
def cos_matrix_multiplication(matrix, vector):
    """
    Calculating pairwise cosine distance using matrix vector multiplication.
    """
    dotted = matrix.dot(vector)
    matrix_norms = np.linalg.norm(matrix, axis=1)
    vector_norm = np.linalg.norm(vector)
    matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
    neighbors = np.divide(dotted, matrix_vector_norms)
    return neighbors

In [59]:
start_time = time.time()
cos_matrix_multiplication(article_embeddings_matrix, article_embeddings_matrix[0])

print ("%s sec"%(time.time() - start_time))

0.16212916374206543 sec


In [65]:
from gensim.models import KeyedVectors, Word2Vec

In [68]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

Using TensorFlow backend.


In [72]:
f = open(directory+'gensim.txt' ,'w', encoding='utf8')
f.write(f'{len(word_to_id)} {128}\n')

word_vecs = article_embeddings_matrix  # W Matrix
for word, idx in word_to_id.items():
    str_vec = ' '.join(map(str, list(word_vecs[idx, :])))
    f.write(f'{word} {str_vec}\n')

f.close()


In [76]:
w2v_cbow = KeyedVectors.load_word2vec_format(directory+'gensim.txt', unicode_errors='ignore')

In [78]:
s = time.time()
w2v_cbow.most_similar(id_to_word[0])
print(time.time() -s)

0.02201700210571289


In [73]:

w2v_cbow = KeyedVectors.load_word2vec_format(directory+'gensim.txt', unicode_errors='ignore')
list(tokenizer.word_index.keys())[:10]
w2v_cbow.most_similar(id_to_word[0])

KeyboardInterrupt: 

In [62]:
from tqdm import tqdm
top_k = 10
cos_sim_list =[]
for i in tqdm(range(len(word_to_id)),mininterval=1):
    cos_sim = cos_matrix_multiplication(article_embeddings_matrix, article_embeddings_matrix[i])
    cos_s = np.asarray(cos_sim)
    cos_sim = cos_s.argsort()[-top_k-1:-1][::-1].tolist()
    cos_sim_list.append(cos_sim)

  0%|                                                                           | 85/505842 [00:17<29:15:57,  4.80it/s]

KeyboardInterrupt: 

  0%|                                                                           | 85/505842 [00:30<29:15:57,  4.80it/s]

# matrix, vecottor 

In [38]:
top_k = 10

In [32]:
import time
start_time = time.time()

import scipy.spatial.distance
result = cos_loop_spatial(article_embeddings_matrix, article_embeddings_matrix[0])
for i in range(len(result)):
    result[i] = 1 - result[i]
print ("%s sec"%(time.time() - start_time))

21.92854332923889 sec


In [36]:
result = np.asarray(result)

In [39]:
result_top_10 = result.argsort()[-top_k -1 : -1][::-1].tolist()

In [46]:
result_top_10_article = [ id_to_word[x] for x in result_top_10]

In [21]:
article_embeddings_matrix[0].shape

(128,)

In [18]:
import time
start_time = time.time()

similarity = np.dot(article_embeddings_matrix,article_embeddings_matrix[0])
square_mag = np.diag(similarity)
inv_square_mag = 1 / square_mag
inv_square_mag[np.isinf(inv_square_mag)] = 0
inv_mag = np.sqrt(inv_square_mag)
cosine = similarity * inv_mag
cosine = cosine.T * inv_mag

print ("%s sec"%(time.time() - start_time))



MemoryError: 

In [ ]:
contents_keyword = []
start_time = time.time()
for value in contents_textrank.values():
    for x in value:
        contents_keyword.append(x)

print ("%s sec"%(time.time() - start_time))
print('text_keyword : ',len(contents_keyword))

# make dictionary
# generate dataset, dictionary for word
word_to_id = {}
id_to_word = {}

for word in contents_keyword:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
# 공백 표시할 어휘 : eos
word_to_id['eos'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'eos'
# vocab에 없는 단어는 UNK로 표시
word_to_id['UNK'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'UNK'

In [6]:
from tqdm import tqdm

In [7]:
# Too slow..
article_by_user = {}

# Using Multiprocessing
from multiprocessing import Pool
article_by_user = {}
for i in tqdm(range(len(test_users_list))):
    article_by_user[i] = extract_article_by_user(test_users_list[i])

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [23:50<00:00,  3.51it/s]


In [20]:
type(article_by_user)

dict

In [21]:
with open(directory+'article_by_testuser.json', 'w', encoding="utf-8") as make_file:
    json.dump(article_by_user, make_file, ensure_ascii=False, indent="\t")

# Multiprocessing Test

In [2]:
import time

start_time = time.time()
def count(name):
    for i in range(1,50000000):
        l = 1
    return 1
num_list = ['p1','p2','p3','p4']
for num in num_list:
    count(num)
    
print ("%s"%(time.time() - start_time))

4.142323017120361


In [1]:
import multiprocessing

In [ ]:
!pip install definitions

In [ ]:
# Fail

In [5]:
import time
from multiprocessing import Pool
import multiprocessing
import defs
start_time = time.time()
def count(name):
    for i in range(1,50000000):
        l = 1
num_list = ['p1','p2','p3','p4']
if __name__ == '__main__':
    pool = Pool()
    results = pool.map(defs.count,numlist)

    
print ("%s"%(time.time() - start_time))

ModuleNotFoundError: No module named 'defs'